Estudos utilizando o dataset Dogs vc. Cats da competição oficial do Kaggle (https://www.kaggle.com/c/dogs-vs-cats).
O Desafio é classificar de forma correta uma imagem de um cachorro ou um gato. Esta tarefa é fácil para um humano, porém é muito difícil para um computador. 
O Dataset contém 25 Mil imagens de dogs e cats.


Utilizaremos deep learning com uma rede convolucional, construída com Keras e utilizando o tensorflow de backend.

Começamos importando as bibliotecas necessárias.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os, shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


Nosso dataset se encontra dentro da pasta -> datasets/dogs-vs-cats, iremos dividir o dataset em “original” que irá conter todo o dataset tanto de treino e teste, criamos também o diretório de trein e validation.

In [2]:
base_dir = 'datasets/dogs-vs-cats'

original_dir = os.path.join(base_dir, 'original')
original_train_dir = os.path.join(original_dir, 'train')

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

cats_train_dir = os.path.join(train_dir, 'cats')
cats_validation_dir = os.path.join(validation_dir, 'cats')

dogs_train_dir = os.path.join(train_dir, 'dogs')
dogs_validation_dir = os.path.join(validation_dir, 'dogs')

Criamos os diretórios definidos no bloco anterior.

In [4]:
if (not os.path.exists(train_dir)):
    os.mkdir(train_dir)
    os.mkdir(validation_dir)
    os.mkdir(cats_train_dir)
    os.mkdir(cats_validation_dir)
    os.mkdir(dogs_train_dir)
    os.mkdir(dogs_validation_dir)
    print('Diretórios criados com sucesso!')

O Bloco a baixo realiza o processo de copias para os diretórios que foram criados.

Serão copiados 10k de imagens de dogs e 10k de cats do diretório "original" para o diretório de "trein".

Serao copiados 2.5k de imagens de dos e 2.5k de imagens de cats para o diretório de "validation".


In [5]:
if not (os.listdir(cats_train_dir) or  os.listdir(dogs_train_dir)):
    def copy_images_to_folder(filename_pattern, start_range, stop_range, src_dir, dst_dir):
        filenames = [filename_pattern.format(i) for i in range(start_range, stop_range)]
        for filename in filenames:
            src = os.path.join(src_dir, filename)
            dst = os.path.join(dst_dir, filename)
            shutil.copyfile(src, dst)

    copy_images_to_folder('cat.{}.jpg', 0, 10000, original_train_dir, cats_train_dir)
    copy_images_to_folder('cat.{}.jpg', 10000, 12500, original_train_dir, cats_validation_dir)

    copy_images_to_folder('dog.{}.jpg', 0, 10000, original_train_dir, dogs_train_dir)
    copy_images_to_folder('dog.{}.jpg', 10000, 12500, original_train_dir, dogs_validation_dir)

O próximo bloco nós definimos um batch_size de 20, este batch será o tamanho do lote que utilizaremos por iteração do modelo. Criamos o "train_datagen" e o "validation_datagen", eles utilizam a função ImageDataGenerator do keras, está função é responsável por gere lotes de dados de imagem do tensor com aumento de dados em tempo real. O próximo passo que o modelo faz é criar o "train_generator" que utiliza a função flow_from_directory, esta função aponta para o train_generator qual é o diretório que estão as imagens de treino e devolve as imagens em formatos de batch, neste caso cada batch de 20. O "validation_generator" faz a mesma coisa que o "train_generator", só que com dados de validação.

In [6]:
batch_size = 20

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(50,50), 
                                                    batch_size=batch_size, 
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_dir, 
                                                              target_size=(50,50), 
                                                              batch_size=batch_size, 
                                                              class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


O Próximo passo é criar nosso modelo utilizando o "Sequential()" do keras, que é uma pilha linear de camadas.

Vamos adicionando nossas camadas uma a uma conforme vamos criando nossa rede profunda. 

A Primeira Camada será do tipo Convolução 2D (Conv2D) o primeiro parâmetro deste layer é a dimensionalidade do espaço de saída, ou seja, o número de filtros de saída da convolução, o parâmetro "kernel_size" recebe a altura e a largura de janela de convolução 2D, o próximo parâmetro "activation" é a função de ativação que neste caso estamos utilizando relu e o último parâmetro "input_shape" recebe uma tupla informando qual a dimensionalidade da nossa imagem (50x50) nos informaremos (50,50,3) por conta da nossa imagem ser 50x50 e RGB.

A Segunda Camada utiliza a função MaxPooling2D, que é responsável por realizar o agrupamento máximo para entradas 2D (imagens), o único parâmetro que estamos utilizando é "pool_size" este parâmetro especifica o tamanho da janela de agrupamento.

Temos uma camada que utiliza a função "Flatten()", ela é responsável por realizar a “normalização” ou "Achatamento/Aplaine" das entradas dos dados para o formato que será utilizando na próxima camada.

A última camada que utilizaremos no nosso modelo é a "Dense" que irá criar uma camada com 512 neurônios ocultos e seu segundo parâmetro é o "activation" que utiliza a função de ativação relu.

No final o processo imprime uma representação resumida do modelo criado utilizando a função summary() do model.


In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 128)         147584    
__________

O próximo passo do nosso processo agora é compilar o nosso modelo, usaremos para isso a função "compile" do nosso modelo, está função recebe como primeiro parâmetro "loss" que é a função de perda, utilizaremos "binary_crossentropy". O segundo parâmetro que passaremos para o compile será "optimizer" que é o parâmetro responsável por tentar otimizar o nosso modelo, usaremos "rmsprop" que é recomendado para redes neurais recorrentes. O último parâmetro que utilizaremos é o "metrics", será a métrica que utilizaremos para medir o quão bom está o nosso algoritmo, no nosso caso a métrica escolhida foi "accuracy".

Agora iremos treinar o nosso modelo com lotes, isto garante uma melhor performance no processo de treino, utilizaremos o "train_generator" que criamos alguns blocos de códigos atrás, ele já está do formato que o nosso fit_generator necessita, o segundo parâmetro é o "steps_pepr_epoch" que é o número total de etapas, estamos utilizando o número total de dados de treino dividido pelo tamanho do nosso batch, o próximo parâmetro é "epochs" que será definido a quantidade de passos do nosso treinamento, o parâmetro "validation_data" recebe o os dados de validação que criamos anteriormente, e  parâmetro "validation_steps" definimos o número total de etapas de processo de validação.


In [8]:
if not (os.path.exists('modelo_treinado.model')):
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    history = model.fit_generator(train_generator, 
                              steps_per_epoch=20000/batch_size, 
                              epochs=10, 
                              validation_data=validation_generator, 
                              validation_steps=5000/batch_size)
    
    model.save('modelo_treinado.model')
else:
    model = tf.keras.models.load_model('modelo_treinado.model')

Epoch 1/10
1000/1000 [==============================] - 210s 210ms/step - loss: 0.6300 - acc: 0.6389 - val_loss: 0.5288 - val_acc: 0.7452
Epoch 2/10
1000/1000 [==============================] - 123s 123ms/step - loss: 0.5205 - acc: 0.7440 - val_loss: 0.4592 - val_acc: 0.7906
Epoch 3/10
1000/1000 [==============================] - 111s 111ms/step - loss: 0.4576 - acc: 0.7866 - val_loss: 0.5075 - val_acc: 0.7424
Epoch 4/10
1000/1000 [==============================] - 118s 118ms/step - loss: 0.4080 - acc: 0.8180 - val_loss: 0.4571 - val_acc: 0.7932
Epoch 5/10
1000/1000 [==============================] - 112s 112ms/step - loss: 0.3751 - acc: 0.8350 - val_loss: 0.4045 - val_acc: 0.8238
Epoch 6/10
1000/1000 [==============================] - 109s 109ms/step - loss: 0.3395 - acc: 0.8554 - val_loss: 0.4577 - val_acc: 0.8030
Epoch 7/10
1000/1000 [==============================] - 109s 109ms/step - loss: 0.3215 - acc: 0.8641 - val_loss: 0.3767 - val_acc: 0.8330
Epoch 8/10
1000/1000 [============

Desta celula para baixo, só estamos fazendo testes manuais em nosso modelo, para isto foi criado uma função para tratar as imagens.

In [9]:
def tratar_img(img):
    img = cv2.imread(img)
    img = cv2.resize(img, (50,50))
    return img.reshape(1,50,50,3)


In [10]:
def predizer(img):
    resultado = model.predict(tratar_img(img))
    if(resultado > 0.5):
        print("O Modelo preveu %s e era %s" % ("Cachorro", img))
    else:
        print("O Modelo preveu %s e era %s" % ("Gato", img))

In [11]:
for i in os.listdir():
    if(i.endswith('.jpg')):
        #image = mpimg.imread(i)
        #plt.imshow(image)
        #plt.show()
        predizer(i)

O Modelo preveu Cachorro e era cachorro.jpg
O Modelo preveu Cachorro e era cachorro1.jpg
O Modelo preveu Cachorro e era cachorro2.jpg
O Modelo preveu Gato e era gato.jpg
O Modelo preveu Gato e era gato1.jpg
